In [26]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import re



""" 네이버 랭킹 뉴스 긁어오기 """
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'}
d_list = []
start_data = 20230101
end_data = 20230113
for date_int in range(start_data, end_data):
    date = str(date_int)
    url = "https://news.naver.com/main/ranking/popularDay.nhn?date=" + date
    html = requests.get(url, headers=headers).text
    soup = BS(html, 'html.parser')
    ranking_total = soup.find_all(class_='rankingnews_box')

    for item in ranking_total:
        media = item.a.strong.text
        news = item.find_all(class_="list_content")
        for new in news:
            d = {}
            d['media'] = media
            d['src'] = "https://news.naver.com/" + new.a['href']
            d['title'] = new.a.text
            d['date'] = date
            d_list.append(d)
df = pd.DataFrame(d_list)

In [27]:
""" 필요 없는 문자 제거 """
def clean_text(row):
    text = row['title']
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("E-mail제거 : " , text , "\n")
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("URL 제거 : ", text , "\n")
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("한글 자음 모음 제거 : ", text , "\n")
    pattern = '<[^>]*>'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("태그 제거 : " , text , "\n")
    pattern = r'\([^)]*\)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("괄호와 괄호안 글자 제거 :  " , text , "\n")
    pattern = '[^\w\s]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("특수기호 제거 : ", text , "\n" )
    pattern = '[^\w\s]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("필요없는 정보 제거 : ", text , "\n" )
    pattern = '["단독"]'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '["속보"]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("단독 속보 제거 : ", text , "\n" )
    text = text.strip()
    # print("양 끝 공백 제거 : ", text , "\n" )
    text = " ".join(text.split())
    # print("중간에 공백은 1개만 : ", text )
    return text

df['title_c'] = df.apply(clean_text, axis=1)

In [28]:
df

,media,src,title,date,title_c
0,매일경제,https://news.naver.com/https://n.news.naver.co...,호스트바 에이스인 남친…“오빠 원래 이런일하는 사람 아니야” [씨네프레소],20230101,호스트바 에이스인 남친오빠 원래 이런일하는 사람 아니야 씨네프레소
1,매일경제,https://news.naver.com/https://n.news.naver.co...,"“사람까지 죽였다”...‘분노폭발’ 중고차, 현대차가 팔면 믿어도 될까 [세상만車]",20230101,사람까지 죽였다분노폭발 중고차 현대차가 팔면 믿어도 될까 세상만車
2,매일경제,https://news.naver.com/https://n.news.naver.co...,‘세계에서 가장 강력한 국가’ 일본 8위…한국은 몇등?,20230101,세계에서 가장 강력한 국가 일본 8위한국은 몇등
3,매일경제,https://news.naver.com/https://n.news.naver.co...,"우리 딸 서울대 합격해 크게 한턱 쐈는데…“엄마, 저 등록 안 할래요” [스물스물]",20230101,우리 딸 서울대 합격해 크게 한턱 쐈는데엄마 저 등록 안 할래요 스물스물
4,매일경제,https://news.naver.com/https://n.news.naver.co...,“쿠키 반죽 같죠? 제 심장이에요”…뉴질랜드女에게 무슨 일이,20230101,쿠키 반죽 같죠 제 심장이에요뉴질랜드女에게 무슨 일이
...,...,...,...,...,...
5035,경기일보,https://news.naver.com/https://n.news.naver.co...,[핫이슈] 집값 ‘뚝’ 미분양 ‘쑥’… 규제 족쇄 풀어도 ‘고금리 발목’,20230112,핫이슈 집값 뚝 미분양 쑥 규제 족쇄 풀어도 고금리 발목
5036,경기일보,https://news.naver.com/https://n.news.naver.co...,‘이재명에 매몰’ 여야 지도부… 인천 현안은 뒷전,20230112,이재명에 매몰 여야 지도부 인천 현안은 뒷전
5037,경기일보,https://news.naver.com/https://n.news.naver.co...,"이재명 “저는 무혐의, 김건희는 명백한 증거 많아”",20230112,이재명 저는 무혐의 김건희는 명백한 증거 많아
5038,경기일보,https://news.naver.com/https://n.news.naver.co...,"인천지검, 900억대 필로폰 밀수 일당 검거 [포토뉴스]",20230112,인천지검 900억대 필로폰 밀수 일당 검거 포토뉴스


In [40]:
""" 키워드 추출 from title """
from konlpy.tag import Kkma
from konlpy.tag import Komoran

kkma = Kkma()
komoran = Komoran()

df['keyword'] = ''
for idx_line in range(len(df)):
    nouns_list = komoran.nouns(df['title_c'].loc[idx_line])
    nouns_list_c = [nouns for nouns in nouns_list if len(nouns) > 1]    # 한글자는 이상한게 많아서 2글자 이상
    a = set(nouns_list_c)
    df.loc[[idx_line], 'keyword'] = a
#df = df[df['media'] != '코리아헤럴드']    # 코리아헤럴드는 영어 제목임
#df = df[df['media'] != '주간경향']    # 주간경향은 같은 title이 많음

In [41]:
df

,media,src,title,date,title_c,keyword
0,매일경제,https://news.naver.com/https://n.news.naver.co...,호스트바 에이스인 남친…“오빠 원래 이런일하는 사람 아니야” [씨네프레소],20230101,호스트바 에이스인 남친오빠 원래 이런일하는 사람 아니야 씨네프레소,"{호스트, 씨네프, 사람, 아니야, 원래, 친오빠, 에이스인}"
1,매일경제,https://news.naver.com/https://n.news.naver.co...,"“사람까지 죽였다”...‘분노폭발’ 중고차, 현대차가 팔면 믿어도 될까 [세상만車]",20230101,사람까지 죽였다분노폭발 중고차 현대차가 팔면 믿어도 될까 세상만車,"{세상, 분노, 중고차, 사람, 폭발, 현대차}"
2,매일경제,https://news.naver.com/https://n.news.naver.co...,‘세계에서 가장 강력한 국가’ 일본 8위…한국은 몇등?,20230101,세계에서 가장 강력한 국가 일본 8위한국은 몇등,"{한국, 국가, 세계, 일본}"
3,매일경제,https://news.naver.com/https://n.news.naver.co...,"우리 딸 서울대 합격해 크게 한턱 쐈는데…“엄마, 저 등록 안 할래요” [스물스물]",20230101,우리 딸 서울대 합격해 크게 한턱 쐈는데엄마 저 등록 안 할래요 스물스물,"{등록, 서울대, 합격, 엄마, 한턱}"
4,매일경제,https://news.naver.com/https://n.news.naver.co...,“쿠키 반죽 같죠? 제 심장이에요”…뉴질랜드女에게 무슨 일이,20230101,쿠키 반죽 같죠 제 심장이에요뉴질랜드女에게 무슨 일이,"{쿠키, 반죽, 뉴질랜드, 심장}"
...,...,...,...,...,...,...
5035,경기일보,https://news.naver.com/https://n.news.naver.co...,[핫이슈] 집값 ‘뚝’ 미분양 ‘쑥’… 규제 족쇄 풀어도 ‘고금리 발목’,20230112,핫이슈 집값 뚝 미분양 쑥 규제 족쇄 풀어도 고금리 발목,"{미분, 핫이슈, 발목, 족쇄, 집값, 고금리, 규제}"
5036,경기일보,https://news.naver.com/https://n.news.naver.co...,‘이재명에 매몰’ 여야 지도부… 인천 현안은 뒷전,20230112,이재명에 매몰 여야 지도부 인천 현안은 뒷전,"{지도부, 이재명, 현안, 여야, 인천, 뒷전, 매몰}"
5037,경기일보,https://news.naver.com/https://n.news.naver.co...,"이재명 “저는 무혐의, 김건희는 명백한 증거 많아”",20230112,이재명 저는 무혐의 김건희는 명백한 증거 많아,"{혐의, 증거, 이재명, 김건희}"
5038,경기일보,https://news.naver.com/https://n.news.naver.co...,"인천지검, 900억대 필로폰 밀수 일당 검거 [포토뉴스]",20230112,인천지검 900억대 필로폰 밀수 일당 검거 포토뉴스,"{일당, 인천지검, 검거, 포토, 밀수, 뉴스}"


In [43]:
from neo4j import GraphDatabase

""" make node & relationship"""
def add_article(tx, title, date, media, keyword):
    tx.run("MERGE (a:Article {title: $title , date: $date, media: $media, keyword: $keyword})",
           title=title, date=date, media=media, keyword=keyword)


def add_media(tx):
    tx.run("MATCH (a:Article) "
           "MERGE (b:Media {name:a.media}) "
           "MERGE (a)<-[r:Publish]-(b)")


def add_keyword(tx):
    tx.run("MATCH (a:Article) "
           "UNWIND a.keyword as k "
           "MERGE (b:Keyword {name:k}) "
           "MERGE (a)-[r:Include]->(b)")



""" 한자와 공백 제거 """
# Neo4j -> Gephi 에서 parsing error의 원인이 될 수 있음
def clean_text_for_neo4j(row):
    text = row['title_c']
    text = re.sub(pattern='[^a-zA-Z0-9ㄱ-ㅣ가-힣]', repl='', string=text)
    # print("영어, 숫자, 한글만 포함 : ", text )
    return text

df['title_c_neo4j'] = df.apply(clean_text_for_neo4j, axis=1)


""" 연결 """
# Neo4j 브라우저에서 설정한 계정의 ID, PASSWORD를 통해 접속
greeter = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "4402"))  


""" 입력 """
# Cyper code를 이용,  크롤링한 Data를 DB에 입력
with greeter.session() as session:
    """ make node """
    for idx in range(len(df)):
        session.write_transaction(add_article, title=df.iloc[idx]['title_c_neo4j'], date=df.iloc[idx]['date'],
                                  media=df.iloc[idx]['media'], keyword=list(df.iloc[idx]['keyword']))
    session.write_transaction(add_media)
    session.write_transaction(add_keyword)

c:\Users\user0216\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: write_transaction has been renamed to execute_write
c:\Users\user0216\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:49: DeprecationWarning: write_transaction has been renamed to execute_write
c:\Users\user0216\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: write_transaction has been renamed to execute_write


In [51]:
df

,media,src,title,date,title_c,keyword,title_c_neo4j
0,매일경제,https://news.naver.com/https://n.news.naver.co...,호스트바 에이스인 남친…“오빠 원래 이런일하는 사람 아니야” [씨네프레소],20230101,호스트바 에이스인 남친오빠 원래 이런일하는 사람 아니야 씨네프레소,"{호스트, 씨네프, 사람, 아니야, 원래, 친오빠, 에이스인}",호스트바에이스인남친오빠원래이런일하는사람아니야씨네프레소
1,매일경제,https://news.naver.com/https://n.news.naver.co...,"“사람까지 죽였다”...‘분노폭발’ 중고차, 현대차가 팔면 믿어도 될까 [세상만車]",20230101,사람까지 죽였다분노폭발 중고차 현대차가 팔면 믿어도 될까 세상만車,"{세상, 분노, 중고차, 사람, 폭발, 현대차}",사람까지죽였다분노폭발중고차현대차가팔면믿어도될까세상만
2,매일경제,https://news.naver.com/https://n.news.naver.co...,‘세계에서 가장 강력한 국가’ 일본 8위…한국은 몇등?,20230101,세계에서 가장 강력한 국가 일본 8위한국은 몇등,"{한국, 국가, 세계, 일본}",세계에서가장강력한국가일본8위한국은몇등
3,매일경제,https://news.naver.com/https://n.news.naver.co...,"우리 딸 서울대 합격해 크게 한턱 쐈는데…“엄마, 저 등록 안 할래요” [스물스물]",20230101,우리 딸 서울대 합격해 크게 한턱 쐈는데엄마 저 등록 안 할래요 스물스물,"{등록, 서울대, 합격, 엄마, 한턱}",우리딸서울대합격해크게한턱쐈는데엄마저등록안할래요스물스물
4,매일경제,https://news.naver.com/https://n.news.naver.co...,“쿠키 반죽 같죠? 제 심장이에요”…뉴질랜드女에게 무슨 일이,20230101,쿠키 반죽 같죠 제 심장이에요뉴질랜드女에게 무슨 일이,"{쿠키, 반죽, 뉴질랜드, 심장}",쿠키반죽같죠제심장이에요뉴질랜드에게무슨일이
...,...,...,...,...,...,...,...
5035,경기일보,https://news.naver.com/https://n.news.naver.co...,[핫이슈] 집값 ‘뚝’ 미분양 ‘쑥’… 규제 족쇄 풀어도 ‘고금리 발목’,20230112,핫이슈 집값 뚝 미분양 쑥 규제 족쇄 풀어도 고금리 발목,"{미분, 핫이슈, 발목, 족쇄, 집값, 고금리, 규제}",핫이슈집값뚝미분양쑥규제족쇄풀어도고금리발목
5036,경기일보,https://news.naver.com/https://n.news.naver.co...,‘이재명에 매몰’ 여야 지도부… 인천 현안은 뒷전,20230112,이재명에 매몰 여야 지도부 인천 현안은 뒷전,"{지도부, 이재명, 현안, 여야, 인천, 뒷전, 매몰}",이재명에매몰여야지도부인천현안은뒷전
5037,경기일보,https://news.naver.com/https://n.news.naver.co...,"이재명 “저는 무혐의, 김건희는 명백한 증거 많아”",20230112,이재명 저는 무혐의 김건희는 명백한 증거 많아,"{혐의, 증거, 이재명, 김건희}",이재명저는무혐의김건희는명백한증거많아
5038,경기일보,https://news.naver.com/https://n.news.naver.co...,"인천지검, 900억대 필로폰 밀수 일당 검거 [포토뉴스]",20230112,인천지검 900억대 필로폰 밀수 일당 검거 포토뉴스,"{일당, 인천지검, 검거, 포토, 밀수, 뉴스}",인천지검900억대필로폰밀수일당검거포토뉴스
